In [1]:
import pandas as pd
import datetime as dt

In [2]:
# get initial data
citi_21 = pd.read_csv('raw/202108.csv')
citi_22 = pd.read_csv('raw/202208.csv')
citi_23 = pd.read_csv('raw/202308.csv')

C:\Users\Peter Solis\AppData\Local\Temp\ipykernel_10472\2815390519.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  citi_21 = pd.read_csv('raw/202108.csv')
C:\Users\Peter Solis\AppData\Local\Temp\ipykernel_10472\2815390519.py:4: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  citi_22 = pd.read_csv('raw/202208.csv')
C:\Users\Peter Solis\AppData\Local\Temp\ipykernel_10472\2815390519.py:5: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  citi_23 = pd.read_csv('raw/202308.csv')


In [22]:
# this is separate so I could run it separately while testing stuff
citi_20 = pd.read_csv('raw/202008.csv')
citi_20['year'] = 2020

In [3]:
citi_21['year'] = 2021
citi_22['year'] = 2022
citi_23['year'] = 2023

In [23]:
# fix columns for 2020, first fix values of customer type
def fix_customer(customer):
    if customer == 'Customer':
        return 'casual'
    elif customer == 'Subscriber':
        return 'member'
    else:
        return customer
citi_20['usertype'] = citi_20['usertype'].apply(fix_customer)
# then fix the time
def fix_time(time):
    new_time = time.split('.')
    return new_time[0]
citi_20['starttime'] = citi_20['starttime'].apply(fix_time)
citi_20['stoptime'] = citi_20['stoptime'].apply(fix_time)
# set gender to none since it's gonna get renamed to be an empty bike type column next
citi_20['gender'] = None
citi_20.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,year
0,2013,2020-08-01 00:00:00,2020-08-01 00:33:34,3295,Central Park W & W 96 St,40.791270,-73.964839,3992,W 169 St & Fort Washington Ave,40.842842,-73.942125,19706,casual,1998,None,2020
1,7002,2020-08-01 00:00:01,2020-08-01 01:56:44,3144,E 81 St & Park Ave,40.776777,-73.959010,3671,E 81 St & 2 Ave,40.774779,-73.954275,37191,member,1966,None,2020
2,155,2020-08-01 00:00:03,2020-08-01 00:02:38,387,Centre St & Chambers St,40.712733,-74.004607,387,Centre St & Chambers St,40.712733,-74.004607,42561,casual,1969,None,2020
3,2720,2020-08-01 00:00:05,2020-08-01 00:45:25,3080,S 4 St & Rodney St,40.709340,-73.956080,3364,Carroll St & 5 Ave,40.675162,-73.981483,41607,member,1990,None,2020
4,2938,2020-08-01 00:00:05,2020-08-01 00:49:03,3510,Adam Clayton Powell Blvd & W 123 St,40.807832,-73.949373,3282,5 Ave & E 88 St,40.783070,-73.959390,36366,casual,1969,None,2020


In [24]:
citi_20.rename(columns = {
    'tripduration': 'ride_id', # this is gonna get deleted, so just using it to cover a missing column
    'gender': 'rideable_type', # renaming this since we need an empty rideable type column for the format when joining
    'starttime': 'started_at',
    'stoptime': 'ended_at',
    'start station name': 'start_station_name',
    'start station id': 'start_station_id',
    'end station name': 'end_station_name',
    'end station id': 'end_station_id',
    'start station latitude': 'start_lat',
    'start station longitude': 'start_lng',
    'end station latitude': 'end_lat',
    'end station longitude': 'end_lng',
    'usertype': 'member_casual',
    'year': 'year'
}, inplace = True)
citi_20_fixed = citi_20[['ride_id','rideable_type','started_at','ended_at','start_station_name','start_station_id','end_station_name','end_station_id','start_lat','start_lng','end_lat','end_lng','member_casual','year']]
citi_20_fixed.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,year
0,2013,None,2020-08-01 00:00:00,2020-08-01 00:33:34,Central Park W & W 96 St,3295,W 169 St & Fort Washington Ave,3992,40.791270,-73.964839,40.842842,-73.942125,casual,2020
1,7002,None,2020-08-01 00:00:01,2020-08-01 01:56:44,E 81 St & Park Ave,3144,E 81 St & 2 Ave,3671,40.776777,-73.959010,40.774779,-73.954275,member,2020
2,155,None,2020-08-01 00:00:03,2020-08-01 00:02:38,Centre St & Chambers St,387,Centre St & Chambers St,387,40.712733,-74.004607,40.712733,-74.004607,casual,2020
3,2720,None,2020-08-01 00:00:05,2020-08-01 00:45:25,S 4 St & Rodney St,3080,Carroll St & 5 Ave,3364,40.709340,-73.956080,40.675162,-73.981483,member,2020
4,2938,None,2020-08-01 00:00:05,2020-08-01 00:49:03,Adam Clayton Powell Blvd & W 123 St,3510,5 Ave & E 88 St,3282,40.807832,-73.949373,40.783070,-73.959390,casual,2020


In [25]:
# combine into df with year columns added above
citi_aug = pd.concat([citi_20_fixed, citi_21, citi_22, citi_23], axis = 0)

In [26]:
# drop IDs, the index is good enough
citi_aug = citi_aug.drop('ride_id', axis = 1)
citi_aug = citi_aug.drop('start_station_id', axis = 1)
citi_aug = citi_aug.drop('end_station_id', axis = 1)
citi_aug.head()

,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,year
0,None,2020-08-01 00:00:00,2020-08-01 00:33:34,Central Park W & W 96 St,W 169 St & Fort Washington Ave,40.791270,-73.964839,40.842842,-73.942125,casual,2020
1,None,2020-08-01 00:00:01,2020-08-01 01:56:44,E 81 St & Park Ave,E 81 St & 2 Ave,40.776777,-73.959010,40.774779,-73.954275,member,2020
2,None,2020-08-01 00:00:03,2020-08-01 00:02:38,Centre St & Chambers St,Centre St & Chambers St,40.712733,-74.004607,40.712733,-74.004607,casual,2020
3,None,2020-08-01 00:00:05,2020-08-01 00:45:25,S 4 St & Rodney St,Carroll St & 5 Ave,40.709340,-73.956080,40.675162,-73.981483,member,2020
4,None,2020-08-01 00:00:05,2020-08-01 00:49:03,Adam Clayton Powell Blvd & W 123 St,5 Ave & E 88 St,40.807832,-73.949373,40.783070,-73.959390,casual,2020


In [27]:
# getting specific time info for trips
dstring = '%Y-%m-%d %H:%M:%S'
citi_aug['started_at'] = pd.to_datetime(citi_aug['started_at'], format = dstring)
citi_aug['ended_at'] = pd.to_datetime(citi_aug['ended_at'], format = dstring)

citi_aug['start_hr'] = citi_aug['started_at'].dt.hour
citi_aug['end_hr'] = citi_aug['ended_at'].dt.hour
citi_aug['start_m'] = citi_aug['started_at'].dt.minute
citi_aug['end_m'] = citi_aug['ended_at'].dt.minute

In [29]:
# calculating rough trip duration and dropping trips that had errors (started 1 day, ended next) or are < 1 minute, as well as time columns we won't use in tableau (this is kinda a fast and dirty way to do it, but we need to get rid of some data to get the filesize down anyway)
citi_aug['trip_duration'] = (citi_aug['end_hr'] * 60 + citi_aug['end_m']) - (citi_aug['start_hr'] * 60 + citi_aug['start_m'])

bad_i = citi_aug[citi_aug['trip_duration'] < 0].index
citi_aug = citi_aug.drop(bad_i)

citi_aug = citi_aug.drop('end_hr', axis = 1)
citi_aug = citi_aug.drop('start_m', axis = 1)
citi_aug = citi_aug.drop('end_m', axis = 1)
citi_aug = citi_aug.drop('started_at', axis = 1)
citi_aug = citi_aug.drop('ended_at', axis = 1)

In [30]:
# split by ride duration
citi_aug_short = citi_aug[citi_aug['trip_duration'] <= 10]
citi_aug_large = citi_aug[citi_aug['trip_duration'] > 10]

In [31]:
# export
citi_aug_short.to_csv('citi_aug_short.csv', index_label='new_id')

In [32]:
citi_aug_large.to_csv('citi_aug_long.csv', index_label='new_id')